In [1]:
import math
import numpy as np
import pandas as pd
import timeit

In [2]:
citiesnp = np.genfromtxt('cities.csv', delimiter=',', skip_header = 1) # load as numpy array

In [3]:
def not_prime(n):
    if n == 2:
        return False
    if n % 2 == 0 or n <= 1:
        return True

    sqr = int(math.sqrt(n)) + 1

    for divisor in range(3, sqr, 2):
        if n % divisor == 0:
            return True
    return False

In [4]:
np_not_prime = np.vectorize(not_prime)

In [5]:
nums = np.arange(0,len(citiesnp))

In [6]:
not_primes_list = nums[np_not_prime(nums)]

In [7]:
def np_total_length(r, c):
    r = np.concatenate(([0], r))
    c = c[r, :]
    cs = np.roll(c, -1, axis =0)
    d = np.sqrt((c[:,1] - cs[:,1])**2 + (c[:,2] - cs[:,2])**2)
    m = np.concatenate((np.arange(1, len(c)+1)[np.newaxis,:],
                        cs[:, 0][np.newaxis,:],
                        d[np.newaxis,:]))
    p = m[:, np.arange(9, len(c), 10)]
    sel = p[0,][np_not_prime(p[1,])].astype(int)
    m[:, sel-1] *= 1.1
    
    return np.sum(m[2])

In [8]:
def np_total_length2(r, c):
    r = np.concatenate(([0], r))
    c = c[r, :]
    cs = np.roll(c, -1, axis =0)
    d = np.sqrt(np.power(c[:,1] - cs[:,1], 2) + np.power(c[:,2] - cs[:,2], 2))
    m = np.concatenate((np.arange(1, len(c)+1)[np.newaxis,:],
                        cs[:, 0][np.newaxis,:],
                        d[np.newaxis,:]))    
    p = m[:, np.arange(9, len(c), 10)]
    sel = p[0,][np_not_prime(p[1,])].astype(int)
    m[:, sel-1] *= 1.1
    
    return np.sum(m[2])

In [9]:
def np_total_length3(r, c):
    r = np.concatenate(([0], r))
    c = c[r, :]
    cs = np.roll(c, -1, axis =0)
    d = np.sqrt(np.power(c[:,1] - cs[:,1], 2) + np.power(c[:,2] - cs[:,2], 2))
    m = np.concatenate((np.arange(1, len(c)+1),
                        cs[:, 0],
                        d)).reshape(len(c), 3, order='F')  
    p = m[np.arange(9, len(c), 10), ]
    sel = p[:,0][np_not_prime(p[:,1])].astype(int)
    m[sel-1] *= 1.1
    
    return np.sum(m[:, 2])

In [10]:
def np_total_length4(r, c, black_list):
    r = np.concatenate(([0], r))
    c = c[r, :]
    cs = np.roll(c, -1, axis =0)
    d = np.sqrt((c[:,1] - cs[:,1])**2 + (c[:,2] - cs[:,2])**2)
    m = np.concatenate((np.arange(1, len(c)+1)[np.newaxis,:],
                        cs[:, 0][np.newaxis,:],
                        d[np.newaxis,:]))
    p = m[:, np.arange(9, len(c), 10)]
    sel = p[0,][np.isin(p[1,], black_list)].astype(int)
    m[:, sel-1] *= 1.1
    
    return np.sum(m[2])

In [11]:
cities_sub = citiesnp[:26]
np.random.seed(3)
r = np.random.permutation(range(1,26))

In [12]:
np_total_length(r, cities_sub)

60566.12398696876

In [13]:
np_total_length4(r, cities_sub, not_primes_list)

60566.12398696876

# Performance

In [14]:
np.random.seed(444)
r = np.random.permutation(range(1, len(citiesnp)))

In [15]:
np_total_length(r, citiesnp)

447590935.07957524

In [16]:
np_total_length2(r, citiesnp)

447590935.07957524

In [17]:
np_total_length3(r, citiesnp)

447590935.07957524

In [18]:
np_total_length4(r, citiesnp, not_primes_list)

447590935.07957524

Previous fastest, numpy version 1:

In [19]:
timeit.timeit('np_total_length(r, citiesnp)', number=1000, globals=globals())

68.2871757880933

New Numpy Version 4:

In [20]:
timeit.timeit('np_total_length4(r, citiesnp, not_primes_list)', number=1000, globals=globals())

38.493459967004455